In [1]:
!pip uninstall transformers -y
!pip install transformers[torch] datasets

Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Successfully uninstalled transformers-4.41.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cubla

In [2]:
# prompt: google mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModel, AutoTokenizer
from transformers import TextClassificationPipeline, BertForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "klue/bert-base"

# 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=60)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델의 구성 정보 확인
config = model.config
print(config.id2label)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6', 7: 'LABEL_7', 8: 'LABEL_8', 9: 'LABEL_9', 10: 'LABEL_10', 11: 'LABEL_11', 12: 'LABEL_12', 13: 'LABEL_13', 14: 'LABEL_14', 15: 'LABEL_15', 16: 'LABEL_16', 17: 'LABEL_17', 18: 'LABEL_18', 19: 'LABEL_19', 20: 'LABEL_20', 21: 'LABEL_21', 22: 'LABEL_22', 23: 'LABEL_23', 24: 'LABEL_24', 25: 'LABEL_25', 26: 'LABEL_26', 27: 'LABEL_27', 28: 'LABEL_28', 29: 'LABEL_29', 30: 'LABEL_30', 31: 'LABEL_31', 32: 'LABEL_32', 33: 'LABEL_33', 34: 'LABEL_34', 35: 'LABEL_35', 36: 'LABEL_36', 37: 'LABEL_37', 38: 'LABEL_38', 39: 'LABEL_39', 40: 'LABEL_40', 41: 'LABEL_41', 42: 'LABEL_42', 43: 'LABEL_43', 44: 'LABEL_44', 45: 'LABEL_45', 46: 'LABEL_46', 47: 'LABEL_47', 48: 'LABEL_48', 49: 'LABEL_49', 50: 'LABEL_50', 51: 'LABEL_51', 52: 'LABEL_52', 53: 'LABEL_53', 54: 'LABEL_54', 55: 'LABEL_55', 56: 'LABEL_56', 57: 'LABEL_57', 58: 'LABEL_58', 59: 'LABEL_59'}


In [6]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load the CSV files
train_df = pd.read_csv('/content/drive/MyDrive/data/train_dataset.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/data/valid_dataset.csv')

# Convert DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

train_dataset = train_dataset.shuffle(seed=42)
valid_dataset = valid_dataset.shuffle(seed=42)

# train_dataset의 절반만 사용하기 위해 데이터셋을 분할
# train_df_half, _ = train_test_split(train_df, test_size=0.5, random_state=42)
# train_dataset_half = Dataset.from_pandas(train_df_half)

# Load tokenizer and model
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
klue_bert = AutoModel.from_pretrained(model_name, num_labels=60)
klue_bert.to(device)

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(examples['hs'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
# train_dataset_half = train_dataset_half.map(tokenize_function, batch_size=True)
# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# train_dataset_half.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


cuda


Map:   0%|          | 0/145949 [00:00<?, ? examples/s]

Map:   0%|          | 0/17965 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
import torch.nn as nn
import torch
from transformers.modeling_outputs import SequenceClassifierOutput

# Define model
class BertCNNClassifier(nn.Module):
    def __init__(self, bert_model, num_labels):
        super(BertCNNClassifier, self).__init__()
        self.bert = bert_model
        self.cnn1 = nn.Conv1d(1536, 512, kernel_size=2, padding=1)
        self.cnn2 = nn.Conv1d(512, 64, kernel_size=2, padding=1)
        self.relu = nn.ReLU()
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(64, num_labels)
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        all_hidden_states = torch.stack(outputs.hidden_states)
        concatenate_pooling = torch.cat((all_hidden_states[-1], all_hidden_states[-4]), dim=-1)
        concatenate_pooling = concatenate_pooling.permute(0, 2, 1)

        x = self.relu(self.cnn1(concatenate_pooling))
        x = self.relu(self.cnn2(x))
        x = self.global_avg_pool(x).squeeze(-1)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [8]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
klue_bert = AutoModel.from_pretrained('klue/bert-base', output_hidden_states=True)

model = BertCNNClassifier(bert_model=klue_bert, num_labels=60)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 예제 입력 텍스트
texts = ["This is a test sentence.", "Another test sentence for checking the model output."]

# 토크나이저를 사용하여 입력 인코딩
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# 모델에 입력 전달하여 출력 얻기
outputs = model(**inputs)

# 출력 확인
logits = outputs.logits
print("Logits shape:", logits.shape)


Logits shape: torch.Size([2, 60])


In [9]:
import logging
import os
import torch
from transformers import EarlyStoppingCallback

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
klue_bert = AutoModel.from_pretrained('klue/bert-base', output_hidden_states=True)

model = BertCNNClassifier(bert_model=klue_bert, num_labels=60)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model.to(device=device)


# Setup logging
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    level=logging.INFO,
    handlers=[
        logging.FileHandler("training.log"),
        logging.StreamHandler()
    ]
)



# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/klue_structure_results_total",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    evaluation_strategy="steps",   # Evaluate based on steps
    eval_steps=400,                # Evaluate every 400 steps
    save_strategy="steps",         # Save based on steps
    save_steps=400,                # Save checkpoint every 400 steps
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/klue_structure_logs_total',  # Directory for storing logs
    logging_steps=10,              # Log every 10 steps
    save_total_limit=None,           # Only last 10 checkpoints are saved
    load_best_model_at_end=True,   # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Use evaluation loss to determine the best model
)

# Initialize Trainer with Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]  # Stop if no improvement after 3 evaluation steps
)

checkpoint_path = "/content/drive/MyDrive/klue_structure_results/checkpoint-19600"

# Fine-tune the model / resume_from_checkpoint=checkpoint_path
trainer.train(resume_from_checkpoint=checkpoint_path)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
400,3.792100,3.740549
800,3.582500,3.486032
1200,3.489500,3.375187
1600,3.301600,3.268320
2000,3.345700,3.193840
2400,3.295700,3.142193
2800,3.158500,3.122931
3200,3.427400,3.103344
3600,3.107600,3.057695
4000,3.217400,3.054344


TrainOutput(global_step=19600, training_loss=2.9518646259697117, metrics={'train_runtime': 17369.5585, 'train_samples_per_second': 42.013, 'train_steps_per_second': 1.313, 'total_flos': 0.0, 'train_loss': 2.9518646259697117, 'epoch': 4.297303222977417})